In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
import warnings
warnings.filterwarnings('ignore')

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from lightgbm import LGBMClassifier

from other_paper.balancers  import MulticlassBalancer

from fmclp.cuae_metric import cuae
from fmclp.get_data import get_data

### LSAC

In [2]:
lsac = get_data('lsac')
lsac

,decile1b,decile3,decile1,cluster,DOB_yr,bar1_yr,bar2_yr,fam_inc,age,parttime,...,attr,target,bar1_P,bar2_P,indxgrp_b 400-460,indxgrp_c 460-520,indxgrp_d 520-580,indxgrp_e 580-640,indxgrp_f 640-700,indxgrp_g 700+
0,10.0,10.0,10.0,1.0,69.0,7.0,94.0,5.0,-62.0,0.0,...,1,2,True,True,False,False,False,False,False,True
1,5.0,4.0,5.0,2.0,69.0,7.0,94.0,4.0,-62.0,0.0,...,1,2,True,True,False,False,False,False,True,False
2,3.0,2.0,3.0,3.0,65.0,7.0,94.0,1.0,-58.0,0.0,...,1,2,True,True,False,False,False,False,False,True
3,7.0,4.0,7.0,3.0,58.0,7.0,94.0,4.0,-51.0,0.0,...,1,2,True,True,False,False,False,False,False,True
4,9.0,8.0,9.0,4.0,68.0,7.0,94.0,4.0,-61.0,0.0,...,1,2,True,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22402,3.0,1.0,3.0,6.0,64.0,2.0,95.0,2.0,-62.0,0.0,...,0,0,False,False,True,False,False,False,False,False
22403,2.0,1.0,3.0,6.0,64.0,7.0,94.0,3.0,-57.0,0.0,...,0,0,False,False,False,False,False,False,False,False
22404,7.0,8.0,7.0,3.0,66.0,7.0,95.0,3.0,-59.0,1.0,...,0,0,True,True,False,False,False,True,False,False
22405,10.0,10.0,10.0,3.0,58.0,7.0,95.0,3.0,-51.0,1.0,...,1,0,True,True,False,False,False,False,True,False


In [4]:
accuracy_list = []
diff_list = []

for i in range(10):     
    y = lsac.drop(['target'], axis=1) 
    x = lsac['target']
    y_train, y_test, x_train, x_test = train_test_split(y, x)
    lsac_classif = LGBMClassifier(verbose=-1)
    lsac_classif.fit(y_train, x_train)
    lsac_preds = lsac_classif.predict(y_test)
 
    a = y_train['attr'].values
    y = x_train.values
    y_ = np.array(lsac_classif.predict(y_train))
        
    lsac_balancer = MulticlassBalancer(y=y, y_=y_, a=a)
    lsac_balancer.adjust(goal='odds')
    lsac_preds = lsac_balancer.predict(y_=np.array(lsac_classif.predict(y_test)), a=y_test['attr'].values)

    accuracy = accuracy_score(lsac_preds, x_test)
    metric = cuae(y_true=x_test, y_pred=lsac_preds, sensitive_features=y_test['attr'].values)['diff']
    
    accuracy_list.append(accuracy)
    diff_list.append(metric)
    print(i+1)

accuracy_list = np.array(accuracy_list)
diff_list = np.array(diff_list)

1
2
3
4
5
6
7
8
9
10


In [6]:
with open("../../results/comparison_results/lsac.txt", 'w') as f:
    f.write(f"""accuracy: {accuracy_list}
diff: {diff_list}
accuracy_mean: {accuracy_list.mean()}
diff_mean: {diff_list.mean()}""")

In [7]:
print(f"""average_accuracy: {accuracy_list.mean()}
average_diff: {diff_list.mean()}""")

average_accuracy: 0.656823967103975
average_diff: 0.17302168104128707
